In [7]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
from math import hypot,sqrt

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [14]:
shape=(64,64)
root_path="/content/drive/My Drive/datasets/cataract dataset/"
train_gen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)
test_gen=ImageDataGenerator(rescale=1/255.0)
train_data_gen=train_gen.flow_from_directory(root_path+'training/',
                                             target_size=shape,
                                             color_mode='grayscale',
                                             class_mode='categorical',
                                             batch_size=8
                                             )
test_data_gen=test_gen.flow_from_directory(root_path+'testing/',
                                               target_size=shape,
                                             color_mode='grayscale',
                                             class_mode='categorical',
                                             batch_size=8)


Found 340 images belonging to 2 classes.
Found 55 images belonging to 2 classes.


In [21]:
# nueral Network
model=Sequential()
model.add(Conv2D(128,(2,2),activation='relu',input_shape=(shape[0],shape[1],1)))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Conv2D(128,(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64,(2,2),activation='relu'))
model.add(Flatten())
model.add(Dense(28,activation='relu'))
model.add(Dense(2,activation='sigmoid'))


In [22]:
opt=optimizers.SGD(learning_rate=0.01,momentum=0.58)
model.compile(optimizer=opt,loss="BinaryCrossentropy",metrics=['accuracy'])
model.fit_generator(train_data_gen,steps_per_epoch=25,epochs=30,validation_data=test_data_gen)

Epoch 1/30
25/25 [==============================] - 1s 37ms/step - loss: 0.7017 - accuracy: 0.6600 - val_loss: 0.6898 - val_accuracy: 0.5273
Epoch 2/30
25/25 [==============================] - 1s 45ms/step - loss: 0.6043 - accuracy: 0.6786 - val_loss: 0.6960 - val_accuracy: 0.5273
Epoch 3/30
25/25 [==============================] - 1s 46ms/step - loss: 0.6228 - accuracy: 0.6939 - val_loss: 0.6866 - val_accuracy: 0.5273
Epoch 4/30
25/25 [==============================] - 1s 39ms/step - loss: 0.4948 - accuracy: 0.7704 - val_loss: 0.6932 - val_accuracy: 0.5273
Epoch 5/30
25/25 [==============================] - 1s 33ms/step - loss: 0.5193 - accuracy: 0.7296 - val_loss: 0.6740 - val_accuracy: 0.5273
Epoch 6/30
25/25 [==============================] - 1s 39ms/step - loss: 0.5206 - accuracy: 0.7245 - val_loss: 0.6706 - val_accuracy: 0.5273
Epoch 7/30
25/25 [==============================] - 1s 37ms/step - loss: 0.4624 - accuracy: 0.7900 - val_loss: 0.6699 - val_accuracy: 0.5273
Epoch 8/30
25

In [24]:
pred=model.predict_generator(test_data_gen,steps=10,use_multiprocessing=True,verbose=1)
np.argmax(pred,axis=1)

10/10 [==============================] - 0s 29ms/step


array([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0])

In [25]:
model.evaluate(test_data_gen)

7/7 [==============================] - 0s 16ms/step - loss: 0.6419 - accuracy: 0.6909


[0.6419159173965454, 0.6909090876579285]

In [28]:
def image_circle(imgR):
	# root_path="/content/drive/My Drive/datasets/cataract dataset/training/cataract/"
	# file=os.path.join(root_path,'eyeimage100.jpg')
	# imgR = cv2.imread(file,1)
	# # print(imgR)
	try:
		img = cv2.cvtColor( imgR, cv2.COLOR_BGR2GRAY)
		# Median Blur at 3
		img = cv2.medianBlur(img,3)
		# Hough Transformation and numpy array circular function tracing
		circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,120,param1=50,param2=50,minRadius=30,maxRadius=0)
		if circles is not None:
			circles = np.uint16(np.around(circles))
			x, y, r = circles[0,:][0]   # Coordinates and Rad
			rows, cols = img.shape
			xr=x
			yr=y
			print (x,y,r)
			# Brightness++
			hsv = cv2.cvtColor( imgR, cv2.COLOR_BGR2HSV)
			h,s,v = cv2.split(hsv)
			v += 250
			final_hsv = cv2.merge ((h,s,v))
			imgR = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
			img = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
			# Crop And Shift Origin : FAILED
			img = img[ y-int(r/2) : y+int(r/2) , x-int(r/2) : x+int(r/2)]
			y = r
			x = r
			rows, cols = img.shape
			# Removing Whitespace
			for i in range(cols):
				for j in range(rows):
					if hypot(i-x, j-y) > r:
						img[j,i] = 0
			cv2.imwrite("iris.jpg",img)
			img=img[30:2*r-30,30:2*r-30]
			y=r-30
			x=r-30
			#img2 = cv2.threshold(img2 , 15 , 125, cv2.THRESH_BINARY)
			circles1 = cv2.HoughCircles(img ,cv2.HOUGH_GRADIENT,1,120,param1=50,param2=50,minRadius=0,maxRadius=0)
			circles1 = np.uint16(np.around(circles1))
			#circles2 = cv2.HoughCircles(img2 ,cv2.HOUGH_GRADIENT,1,120,param1=50,param2=50,minRadius=1,maxRadius=0)
			#circles2 = np.uint16(np.around(circles2))
			x2, y2, r2 = circles1[0,:][0]
			rows2, cols2,__ = imgR.shape
			xn=x2+30+xr-r
			yn=y2+30+yr-r
			# print x2,y2,r2
			npixels=0
			intensitySum=0
			bgrList=[]
			for i in range(cols2):
				for j in range(rows2):
					if hypot(i-xn, j-yn) < r2:
						npixels+=1
						b=imgR[j,i][0]
						g=imgR[j,i][1]
						r=imgR[j,i][2]
						#calculate intensity
						intensitySum+=b
						intensitySum+=g
						intensitySum+=r
						# #calculate unformity

						#caculate deviation
						bgrList.append(b)
						bgrList.append(g)
						bgrList.append(r)

			intensity = (intensitySum*1.0)/(3.0*npixels)
			devSum=0
			for f in bgrList:
				devSum+=(f-intensity)*(f-intensity)
			devSum=sqrt(devSum)
			devSum=devSum*1.0/(81*sqrt(((2*r2)*(2*r2))-1))
			print( devSum,intensity)
		else:
			print("No circle found in image")
		#plt.imshow(img2, cmap = 'gray', interpolation = 'bicubic')
		# plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
		# plt.xticks([]), plt.yticks([])
		# plt.show()
	except:
		return 0
	return img

In [26]:
model.save('/content/drive/My Drive/Cataract.h5')

In [29]:
from keras.models import load_model

import numpy as np

model_new = load_model('/content/drive/My Drive/Cataract.h5')

model_new.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


In [30]:
import cv2
# Load the cascade
eye_cascade = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_eye.xml')
# Read the input image
path='/content/drive/My Drive/datasets/cataract dataset/testing/cataract/'
l=len(os.listdir(path))
plt.figure(figsize=(20, 20))
for i,img_ in enumerate(os.listdir(path)):
  img = cv2.imread(os.path.join(path,img_),1)
  img=image_circle(img)
  try:
    img2=img
    img2=np.array(img2)/255.0
    img2.resize((32,32))
    img2=img2.reshape((1,32,32,1))
    classes = model_new.predict_classes(img2)
    if classes[-1]==0:
      classes='non cataract'
    else:
      classes='cataract'
    # Display the output
    plt.subplot(int(l/3)+1,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img,cmap='gray')
    plt.xlabel(classes)
  except:
    pass
plt.show()

126 66 66
152 94 82
162 56 39
144 92 90
254 256 250
0.3271412627396265 126.66856840993132
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
168 68 65
96 94 95
254 256 250
0.3271412627396265 126.66856840993132
432 432 462
200 94 114
108 92 68
396 394 425
0.8549434974000957 138.99102802937838
66 74 33
276 274 305
0.8366108602317877 95.52586831165348
244 198 195
0.5076757037272108 42.51321552635554
94 94 85
116 118 104
274 272 300
0.667984812391184 148.29424424700625
114 114 113
156 140 59
126 122 123
No circle found in image
No circle found in image
96 114 79
114 102 101
432 432 462
334 342 363
68 66 40
150 170 150
96 90 87
88 156 104
No circle found in image


<Figure size 1440x1440 with 0 Axes>

In [32]:
import cv2
from google.colab.patches import cv2_imshow
face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_eye.xml')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img = cap.read()
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
      cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
      roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_color)
    for (ex,ey,ew,eh) in eyes:
      cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    cv2_imshow(img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()